In [1]:
%%HTML
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [2]:
#cell 1
from astropy.time import Time
import pandas as pd
from astropy.utils import iers
import matplotlib.pyplot as plt
iers.IERS_A_URL  ='ftp://cddis.gsfc.nasa.gov/pub/products/iers/finals2000A.all'
import yaml
import sys
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML, display, clear_output, Javascript
from plotly import graph_objs as go
from IPython.display import Image
import os
from datetime import date
import datetime
from astropy import units as u
import datetime
from astroplan import Observer
%matplotlib notebook
from astropy.coordinates import SkyCoord,EarthLocation,CartesianRepresentation
import markdown 
from ipywidgets import HTML
from SPOCK_chilean.make_night_plans import chilean_time

%gui asyncio

In [3]:
#cell 2
# WIDGETS ---------------------------------------------------------------------------------------------
chilean_ID_widget = widgets.Text(description='ID for ESO',placeholder='ID')
 
chilean_title_widget = widgets.Text(description='Title',placeholder='title')

chilean_PI_widget = widgets.Text(description='PI',placeholder='PI')

start_date_widget =widgets.DatePicker(description='Start date',disabled=False)

chilean_nb_target_widget = widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='# targets:',disabled=False)

telescope_widget = widgets.Combobox(placeholder='SSO telescope',options=['Io','Europa','Ganymede','Callisto'],
    description='Telescope:',ensure_option=True,disabled=False)

run_widget = widgets.Button(value=False,description='Click me',disabled=False,
    button_style='',tooltip='Description',icon='check')

def run_all(b): 
    display(Javascript('Jupyter.notebook.execute_cells([3])'))

run_check_widget = widgets.Button(value=False,description='Check Plans',disabled=False,
    button_style='warning',tooltip='Check',icon='fa-arrow-right')
    
def run_check(b): 
    try:
        date_widget = datetime.datetime.combine(start_date_widget.value, datetime.time(1,0,0))
        date = Time(date_widget,format='datetime',out_subfmt='date')
        Path = './Plans_by_date/' + date.iso
        if os.path.exists(Path):
            print('INFO: Plan for on the ' + date.iso + ' are:')
            for rr, dd, ff in os.walk(Path):
                print(ff)
            display(Javascript('Jupyter.notebook.execute_cells([5])'))

        else:
            print('ERROR: No known plans for this date ')
    except TypeError: 
        print('ERROR: Please enter a correct date ')


#---------------------------------------------------------------------------------------------

chilean_tab = widgets.Accordion(children=[widgets.HBox([chilean_ID_widget,chilean_title_widget,chilean_PI_widget]),\
                                          widgets.VBox([widgets.HBox([start_date_widget,chilean_nb_target_widget,telescope_widget])]),\
                                          run_widget])
chilean_tab.set_title(0, 'Requiered')
chilean_tab.set_title(1, 'Plan maker options')
chilean_tab.set_title(2, 'Go to plan maker')

#---------------------------------------------------------------------------------------------

check_tab = widgets.Accordion(children=[widgets.VBox([start_date_widget,run_check_widget])])
check_tab.set_title(0, 'Plan checker')
run_widget.on_click(run_all)
run_check_widget.on_click(run_check)

tab_nest = widgets.Tab()
tab_nest.children = [chilean_tab,check_tab]
tab_nest.set_title(0, 'Plan Maker ')
tab_nest.set_title(1, 'Plan Checker ')
tab_nest

<IPython.core.display.Javascript object>

In [8]:
#cell 3
if start_date_widget.value is None:
    #sys.exit('ERROR: You must choose a date')
    Javascript('Jupyter.notebook.execute_cells([6])')
else:
    chilean_target_name_widget = [widgets.Text(description='Target name',placeholder='Chxxxx-xxxx'),
                                  widgets.Text(description='Target name',placeholder='Chxxxx-xxxx'),
                                  widgets.Text(description='Target name',placeholder='Chxxxx-xxxx'),
                                  widgets.Text(description='Target name',placeholder='Chxxxx-xxxx'),
                                  widgets.Text(description='Target name',placeholder='Chxxxx-xxxx'),
                                  widgets.Text(description='Target name',placeholder='Chxxxx-xxxx'),
                                  widgets.Text(description='Target name',placeholder='Chxxxx-xxxx'),
                                  widgets.Text(description='Target name',placeholder='Chxxxx-xxxx'),
                                  widgets.Text(description='Target name',placeholder='Chxxxx-xxxx'),
                                  widgets.Text(description='Target name',placeholder='Chxxxx-xxxx')]

    chilean_filter_widget = [widgets.Combobox(placeholder='Choose a filter',options=['I+z','r','z','i','Clear'],
        description='Filter :',ensure_option=True,disabled=False),
        widgets.Combobox(placeholder='Choose a filter',options=['I+z','r','z','i','Clear'],
        description='Filter :',ensure_option=True,disabled=False),
        widgets.Combobox(placeholder='Choose a filter',options=['I+z','r','z','i','Clear'],
        description='Filter :',ensure_option=True,disabled=False),
        widgets.Combobox(placeholder='Choose a filter',options=['I+z','r','z','i','Clear'],
        description='Filter :',ensure_option=True,disabled=False),
        widgets.Combobox(placeholder='Choose a filter',options=['I+z','r','z','i','Clear'],
        description='Filter :',ensure_option=True,disabled=False),
        widgets.Combobox(placeholder='Choose a filter',options=['I+z','r','z','i','Clear'],
        description='Filter :',ensure_option=True,disabled=False),
        widgets.Combobox(placeholder='Choose a filter',options=['I+z','r','z','i','Clear'],
        description='Filter :',ensure_option=True,disabled=False),
        widgets.Combobox(placeholder='Choose a filter',options=['I+z','r','z','i','Clear'],
        description='Filter :',ensure_option=True,disabled=False),
        widgets.Combobox(placeholder='Choose a filter',options=['I+z','r','z','i','Clear'],
        description='Filter :',ensure_option=True,disabled=False),
        widgets.Combobox(placeholder='Choose a filter',options=['I+z','r','z','i','Clear'],
        description='Filter :',ensure_option=True,disabled=False)]

    chilean_texp_widget = [widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
        widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
                          widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
        widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
                          widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
        widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
                          widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
        widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
                          widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
        widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
                          widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
        widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
                          widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
        widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
                          widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
        widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
                          widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
        widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
                          widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False),
        widgets.BoundedIntText(value=20,min=10,max=2000,step=1,description='Exp time:',disabled=False)]

    chilean_binnig_widget = [widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
        widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
                            widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
        widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
                            widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
        widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
                            widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
        widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
                            widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
        widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
                            widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
        widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
                            widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
        widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
                            widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
        widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
                            widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
        widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
                            widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False),
        widgets.BoundedIntText(value=1,min=1,max=10,step=1,description='binning:',disabled=False)]

    chilean_ra_widgget = [widgets.BoundedFloatText(value=0.0000,min=0.0,max=360,step=0.0001,description='RA (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=0.0,max=360,step=0.0001,description='RA (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=0.0,max=360,step=0.0001,description='RA (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=0.0,max=360,step=0.0001,description='RA (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=0.0,max=360,step=0.0001,description='RA (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=0.0,max=360,step=0.0001,description='RA (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=0.0,max=360,step=0.0001,description='RA (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=0.0,max=360,step=0.0001,description='RA (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=0.0,max=360,step=0.0001,description='RA (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=0.0,max=360,step=0.0001,description='RA (in °):',disabled=False)]

    chilean_dec_widgget = [widgets.BoundedFloatText(value=0.0000,min=-90.0,max=+90.0,step=0.0001,description='DEC (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=-90.0,max=+90.0,step=0.0001,description='DEC (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=-90.0,max=+90.0,step=0.0001,description='DEC (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=-90.0,max=+90.0,step=0.0001,description='DEC (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=-90.0,max=+90.0,step=0.0001,description='DEC (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=-90.0,max=+90.0,step=0.0001,description='DEC (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=-90.0,max=+90.0,step=0.0001,description='DEC (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=-90.0,max=+90.0,step=0.0001,description='DEC (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=-90.0,max=+90.0,step=0.0001,description='DEC (in °):',disabled=False),
        widgets.BoundedFloatText(value=0.0000,min=-90.0,max=+90.0,step=0.0001,description='DEC (in °):',disabled=False)]

    chilean_count_widget = [widgets.BoundedIntText(value=5000,min=1,max=10000,step=1,description='Counts:',disabled=False),
        widgets.BoundedIntText(value=5000,min=1,max=10000,step=1,description='Counts:',disabled=False),
        widgets.BoundedIntText(value=5000,min=1,max=10000,step=1,description='Counts:',disabled=False),
        widgets.BoundedIntText(value=5000,min=1,max=10000,step=1,description='Counts:',disabled=False),
        widgets.BoundedIntText(value=5000,min=1,max=10000,step=1,description='Counts:',disabled=False),
        widgets.BoundedIntText(value=5000,min=1,max=10000,step=1,description='Counts:',disabled=False),
        widgets.BoundedIntText(value=5000,min=1,max=10000,step=1,description='Counts:',disabled=False),
        widgets.BoundedIntText(value=5000,min=1,max=10000,step=1,description='Counts:',disabled=False),
        widgets.BoundedIntText(value=5000,min=1,max=10000,step=1,description='Counts:',disabled=False),
        widgets.BoundedIntText(value=5000,min=1,max=10000,step=1,description='Counts:',disabled=False)]
    
    chilean_gaia_widget = [widgets.Text(description='Gaia_ID',placeholder='ID'),
                          widgets.Text(description='Gaia ID',placeholder='ID'),
                          widgets.Text(description='Gaia ID',placeholder='ID'),
                          widgets.Text(description='Gaia ID',placeholder='ID'),
                          widgets.Text(description='Gaia ID',placeholder='ID'),
                          widgets.Text(description='Gaia ID',placeholder='ID'),
                          widgets.Text(description='Gaia ID',placeholder='ID'),
                          widgets.Text(description='Gaia ID',placeholder='ID'),
                          widgets.Text(description='Gaia ID',placeholder='ID'),
                          widgets.Text(description='Gaia ID',placeholder='ID')]
    
    chilean_Jmag_widget = [widgets.BoundedFloatText(value=10.00,min=1.00,max=20.00,step=0.01,description='Jmag:',disabled=False),
        widgets.BoundedFloatText(value=10.00,min=1.00,max=20.00,step=0.01,description='Jmag:',disabled=False),
        widgets.BoundedFloatText(value=10.00,min=1.00,max=20.00,step=0.01,description='Jmag:',disabled=False),
        widgets.BoundedFloatText(value=10.00,min=1.00,max=20.00,step=0.01,description='Jmag:',disabled=False),
        widgets.BoundedFloatText(value=10.00,min=1.00,max=20.00,step=0.01,description='Jmag:',disabled=False),
        widgets.BoundedFloatText(value=10.00,min=1.00,max=20.00,step=0.01,description='Jmag:',disabled=False),
        widgets.BoundedFloatText(value=10.00,min=1.00,max=20.00,step=0.01,description='Jmag:',disabled=False),
        widgets.BoundedFloatText(value=10.00,min=1.00,max=20.00,step=0.01,description='Jmag:',disabled=False),
        widgets.BoundedFloatText(value=10.00,min=1.00,max=20.00,step=0.01,description='Jmag:',disabled=False),
        widgets.BoundedFloatText(value=10.00,min=1.00,max=20.00,step=0.01,description='Jmag:',disabled=False)]
 
    chilean_SpT_widget = [widgets.Combobox(placeholder='Choose a SpT',options=['B0','B1','B3','B6','B8','A0','A2','A3','A5','F0','F2','F5'
                    ,'F8','G0','G1','G2','G5','G8','K0','K2','K5','K7','M0','M2','M4','M5','M6','M7'
                    ,'M8','M9','L2','L5','L8'],description='Spectral type :',ensure_option=True,disabled=False),
                        widgets.Combobox(placeholder='Choose a SpT',options=['B0','B1','B3','B6','B8','A0','A2','A3','A5','F0','F2','F5'
                    ,'F8','G0','G1','G2','G5','G8','K0','K2','K5','K7','M0','M2','M4','M5','M6','M7'
                    ,'M8','M9','L2','L5','L8'],description='Spectral type :',ensure_option=True,disabled=False),
                         widgets.Combobox(placeholder='Choose a SpT',options=['B0','B1','B3','B6','B8','A0','A2','A3','A5','F0','F2','F5'
                    ,'F8','G0','G1','G2','G5','G8','K0','K2','K5','K7','M0','M2','M4','M5','M6','M7'
                    ,'M8','M9','L2','L5','L8'],description='Spectral type :',ensure_option=True,disabled=False),
                         widgets.Combobox(placeholder='Choose a SpT',options=['B0','B1','B3','B6','B8','A0','A2','A3','A5','F0','F2','F5'
                    ,'F8','G0','G1','G2','G5','G8','K0','K2','K5','K7','M0','M2','M4','M5','M6','M7'
                    ,'M8','M9','L2','L5','L8'],description='Spectral type :',ensure_option=True,disabled=False),
                         widgets.Combobox(placeholder='Choose a SpT',options=['B0','B1','B3','B6','B8','A0','A2','A3','A5','F0','F2','F5'
                    ,'F8','G0','G1','G2','G5','G8','K0','K2','K5','K7','M0','M2','M4','M5','M6','M7'
                    ,'M8','M9','L2','L5','L8'],description='Spectral type :',ensure_option=True,disabled=False),
                         widgets.Combobox(placeholder='Choose a SpT',options=['B0','B1','B3','B6','B8','A0','A2','A3','A5','F0','F2','F5'
                    ,'F8','G0','G1','G2','G5','G8','K0','K2','K5','K7','M0','M2','M4','M5','M6','M7'
                    ,'M8','M9','L2','L5','L8'],description='Spectral type :',ensure_option=True,disabled=False),
                         widgets.Combobox(placeholder='Choose a SpT',options=['B0','B1','B3','B6','B8','A0','A2','A3','A5','F0','F2','F5'
                    ,'F8','G0','G1','G2','G5','G8','K0','K2','K5','K7','M0','M2','M4','M5','M6','M7'
                    ,'M8','M9','L2','L5','L8'],description='Spectral type :',ensure_option=True,disabled=False),
                         widgets.Combobox(placeholder='Choose a SpT',options=['B0','B1','B3','B6','B8','A0','A2','A3','A5','F0','F2','F5'
                    ,'F8','G0','G1','G2','G5','G8','K0','K2','K5','K7','M0','M2','M4','M5','M6','M7'
                    ,'M8','M9','L2','L5','L8'],description='Spectral type :',ensure_option=True,disabled=False),
                         widgets.Combobox(placeholder='Choose a SpT',options=['B0','B1','B3','B6','B8','A0','A2','A3','A5','F0','F2','F5'
                    ,'F8','G0','G1','G2','G5','G8','K0','K2','K5','K7','M0','M2','M4','M5','M6','M7'
                    ,'M8','M9','L2','L5','L8'],description='Spectral type :',ensure_option=True,disabled=False),
                         widgets.Combobox(placeholder='Choose a SpT',options=['B0','B1','B3','B6','B8','A0','A2','A3','A5','F0','F2','F5'
                    ,'F8','G0','G1','G2','G5','G8','K0','K2','K5','K7','M0','M2','M4','M5','M6','M7'
                    ,'M8','M9','L2','L5','L8'],description='Spectral type :',ensure_option=True,disabled=False)]

    chilean_start_time_widget = [widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00'),\
                                widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00'),
                                widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00'),
                                widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00'),
                                widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00'),
                                widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00'),
                                widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00'),
                                widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00'),
                                widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00'),
                                widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00')]
    
    chilean_end_time_widget = [widgets.Text(description='End time',placeholder='2020-01-17 03:00:00'),\
                              widgets.Text(description='End time',placeholder='2020-01-17 03:00:00'),
                              widgets.Text(description='End time',placeholder='2020-01-17 03:00:00'),
                              widgets.Text(description='End time',placeholder='2020-01-17 03:00:00'),
                              widgets.Text(description='End time',placeholder='2020-01-17 03:00:00'),
                              widgets.Text(description='End time',placeholder='2020-01-17 03:00:00'),
                              widgets.Text(description='End time',placeholder='2020-01-17 03:00:00'),
                              widgets.Text(description='End time',placeholder='2020-01-17 03:00:00'),
                              widgets.Text(description='End time',placeholder='2020-01-17 03:00:00'),
                              widgets.Text(description='End time',placeholder='2020-01-17 03:00:00')]


    run_chilean_widget = widgets.Button(value=False,description='Click me',disabled=False,
        button_style='',tooltip='Description',icon='check')

    run_etc_widget = widgets.Button(value=False,description='ETC SPECULOOS',disabled=False,
        button_style='info',tooltip='Open ETC',icon='fa-arrow-right')
    
    def run_etc(b):
        date_widget = datetime.datetime.combine(start_date_widget.value, datetime.time(1,0,0))
        date = Time(date_widget,format='datetime',out_subfmt='date')
        def etc_chilean(Jmag,SpT,Filter):
            a = chilean_time(date.iso)
            texp, peak, SNR = a.etc_chilean(Jmag,SpT,Filter)
            return 'INFO: The exopusre time we recommend is ' + str(int(texp)*u.s) + ', Peak is: ' + str(round(peak,3)) + \
        ' ADU, and SNR is:  ' + str(round(SNR,3))
        interact(etc_chilean,Jmag=widgets.BoundedFloatText(value=10.00,min=1.00,max=20.00,\
        step=0.01,description='Jmag:',disabled=False),SpT=['B0','B1','B3','B6','B8', \
                        'A0','A2','A3','A5', \
                        'F0','F2','F5','F8', \
                        'G0','G1','G2','G5','G8', \
                        'K0','K2','K5','K7', \
                        'M0','M2','M4','M5','M6','M7','M8','M9', \
                        'L2','L5','L8'],Filter=['I+z','z','i','Clear'])

    def run_chilean(b): 
        display(Javascript('Jupyter.notebook.execute_cells([4])'))


    chilean_tab = []

    for i in range(0,chilean_nb_target_widget.value):
        chilean_tab.append(widgets.Accordion(children=[widgets.VBox([chilean_target_name_widget[i],widgets.HBox([chilean_ra_widgget[i],chilean_dec_widgget[i]]),
                                                  widgets.HBox([chilean_filter_widget[i],chilean_texp_widget[i],chilean_binnig_widget[i]]),\
                                                  widgets.HBox([chilean_start_time_widget[i],chilean_end_time_widget[i]]),\
                                                  widgets.HBox([chilean_count_widget[i],chilean_SpT_widget[i],chilean_Jmag_widget[i]]),run_etc_widget])]))
        chilean_tab[i].set_title(0,' Field #' + str(i+1))
    chilean_tab.append(widgets.Accordion(children=[run_chilean_widget]))
    chilean_tab[len(chilean_tab)-1].set_title(0, 'Run plan maker')

    run_chilean_widget.on_click(run_chilean)
    run_etc_widget.on_click(run_etc)
    w = run_etc_widget
    tab_nest = widgets.Tab()
    tab_nest.children = [widgets.VBox([chilean_tab[i] for i in range(len(chilean_tab))])]
    tab_nest.set_title(0, 'Plan maker ')
    display(tab_nest)

interactive(children=(BoundedFloatText(value=10.0, description='Jmag:', max=20.0, min=1.0, step=0.01), Dropdow…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
#cell 4
run_check_widget = widgets.Button(
    value=False,
    description='Check Plans',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Check',
    icon='fa-arrow-right')
    
def run_check(b): 
        display(Javascript('Jupyter.notebook.execute_cells([5])'))
        
try: 
    date_widget = datetime.datetime.combine(start_date_widget.value, datetime.time(1,0,0))
    date = Time(date_widget,format='datetime',out_subfmt='date')
    date_folder = Time(date_widget,format='datetime',out_subfmt='date').iso
    try:
        for i in range(0,chilean_nb_target_widget.value):
            if i ==0:
                target_chilean = pd.DataFrame({'Date': date, 'Telescope': telescope_widget.value, 'Name': chilean_target_name_widget[0].value,
                                       'Start': Time(chilean_start_time_widget[0].value).iso, 'End': Time(chilean_end_time_widget[0].value).iso,
                                       'RA': chilean_ra_widgget[0].value , \
                                       'DEC': chilean_dec_widgget[0].value, 'Filter': chilean_filter_widget[0].value,
                                       'texp': chilean_texp_widget[0].value, \
                                       'Counts': chilean_count_widget[0].value, '#target': 0,'Gaia_ID':'gaia_id','Jmag':chilean_Jmag_widget[0].value,'SpT':chilean_SpT_widget[0].value},
                                      index=[0])
            if i > 0:
                other_target = {'Date': date, 'Telescope': telescope_widget.value, 'Name': chilean_target_name_widget[i].value,\
                                   'Start': Time(chilean_start_time_widget[i].value).iso, 'End': Time(chilean_end_time_widget[i].value).iso,\
                                   'RA': chilean_ra_widgget[i].value ,\
                                   'DEC': chilean_dec_widgget[i].value, 'Filter': chilean_filter_widget[i].value,\
                                   'texp': chilean_texp_widget[i].value,\
                                   'Counts': chilean_count_widget[i].value, '#target': 0,'Gaia_ID':'gaia_id','Jmag':chilean_Jmag_widget[i].value,'SpT':chilean_SpT_widget[i].value}

                target_chilean = target_chilean.append(other_target, ignore_index=True, sort=False)

        a = chilean_time(date.iso)
        a.make_plans(target_chilean)
        display(target_chilean)
        display(run_check_widget)
        run_check_widget.on_click(run_check)

    except ValueError:
        print('ERROR: Please ensure you have filled in all the fields')

except TypeError:
    print('ERROR: Chose a date')

INFO: File for date 2020-06-25 exists, overwriting


,Date,Telescope,Name,Start,End,RA,DEC,Filter,texp,Counts,#target,Gaia_ID,Jmag,SpT
0,2020-06-25 01:00:00,Europa,Ch3,2020-06-26 06:00:00.000,2020-06-26 07:00:00.000,346.0,-5.04,I+z,23,5000,0,gaia_id,10.0,


Button(button_style='warning', description='Check Plans', icon='arrow-right', style=ButtonStyle(), tooltip='Ch…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
#cell 5
try:
    date_widget = datetime.datetime.combine(start_date_widget.value, datetime.time(1,0,0))
    date = Time(date_widget,format='datetime',out_subfmt='date')
    a = chilean_time(date.iso)
    a.check_plans()

except TypeError:
    print('ERROR: Chose a date')


INFO: OK, filter chosen for Ch3 is in the list
INFO: OK, exposure time chosen for Ch3 is >10s
INFO: OK, field Ch3 respects the elevation constraint
INFO: OK, field Ch3 respects the moon constraint
INFO: Ok, field Ch3 is scheduled for more than 15 min


ReadTimeout: HTTPSConnectionPool(host='mast.stsci.edu', port=443): Read timed out. (read timeout=None)

In [7]:
#cell 6
print('ERROR: Chose a date')

ERROR: Chose a date
